# Import

In [12]:
import torch 
import torchvision.transforms as T 
import torch.nn.init
from torch.utils.data import DataLoader
from torchvision import datasets
from torch import nn
from tqdm import tqdm
import matplotlib.pyplot as plt

In [13]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# 랜덤 시드 고정
torch.manual_seed(777)

# GPU 사용 가능일 경우 랜덤 시드 고정
if device == "cuda": 
    torch.cuda.manual_seed_all(777)
    
learning_rate = 1e-3
training_epochs = 1000
batch_size = 100

# Data Load

In [14]:
custom_train_dataset = datasets.ImageFolder(root="../MNIST/mset/train/", transform=T.Compose([T.Grayscale(1),
                                                                                           #T.RandomAffine(degrees=(5,5), translate=(0.1, 0.15), scale=(0.85, 0.95)),
                                                                                           #T.RandomRotation(10.),
                                                                                           T.ToTensor()
                                                                                           ]))

custom_test_dataset = datasets.ImageFolder(root="../MNIST/mset/test/", transform=T.Compose([T.Grayscale(1),
                                                                                         T.ToTensor()
                                                                                         ]))

train_loader = DataLoader(dataset=custom_train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True,
                                           drop_last=True)

test_loader = DataLoader(dataset=custom_test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False,
                                          drop_last=False)

# Data visualization

In [15]:
print(f"train size: {len(custom_train_dataset)}")
print(f"test size: {len(custom_test_dataset)}")
print("train_dataset_classes_names = ", custom_train_dataset.class_to_idx)
print("num classes = ",len(custom_train_dataset.class_to_idx))

examples = enumerate(test_loader)
batch_idx, (example_data, example_targets) = next(examples)

example_data.shape

train size: 22195
test size: 4571
train_dataset_classes_names =  {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8, 'j': 9, 'k': 10, 'l': 11, 'm': 12, 'n': 13, 'o': 14, 'p': 15, 'q': 16, 'r': 17, 's': 18, 't': 19, 'u': 20, 'v': 21, 'w': 22, 'x': 23, 'y': 24, 'z': 25}
num classes =  26


torch.Size([100, 1, 28, 28])

# Model

In [18]:
class CNN(torch.nn.Module):
    def __init__(self): 
        super(CNN, self).__init__()
        # L1 ImgIn shape=(?, 28, 28, 1)
        # Conv -> (?, 28, 28, 128)
        # Pool -> (?, 14, 14, 128)
        self.layer1 = self.conv1(1, 128)
        
        # L2 ImgIn shape=(?, 14, 14, 128)
        # Conv ->(?, 14, 14, 256)
        # Pool ->(?, 7, 7, 256)
        self.layer2 = self.conv2(128, 256)
        
        # L3 ImgIn shape=(?, 7, 7, 256)
        # Conv ->(?, 7, 7, 512)
        # Pool ->(?, 4, 4, 512)
        self.layer3 = self.conv3(256, 512)
        
        # L4 ImgIn shape=(?, 4, 4, 512)
        # Conv ->(?, 4, 4, 26)
        # Pool ->(?, 1, 1, 26)
        self.gap = self.global_avg_pool(512, 26)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.gap(out)
        out = out.view(-1, 26)
        return out

    def conv1(self, in_ch, out_ch):
        return nn.Sequential(
            nn.Conv2d(in_ch, out_ch, kernel_size=5, padding=2),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True))
    
    def conv2(self, in_ch, out_ch):
        return nn.Sequential(
            nn.Conv2d(in_ch, out_ch, kernel_size=5, padding=2),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Dropout2d(p=0.4))
           
    
    def conv3(self, in_ch, out_ch):
        return nn.Sequential(
            nn.Conv2d(in_ch, out_ch, kernel_size=5, padding=2),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Dropout2d(p=0.4))
            
     
    def global_avg_pool(self, in_ch, out_ch):
        return nn.Sequential(
            nn.Conv2d(in_ch, out_ch, kernel_size=5,padding=2),
            nn.AdaptiveAvgPool2d(1),
            nn.BatchNorm1d(out_ch),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.4))
            

# CNN model, loss, optim, scheduler

In [19]:
model = CNN().to(device)
#model = torch.load("../weight/Gap_best.pth")

criterion = torch.nn.CrossEntropyLoss().to(device) # 비용 함수에 소프트맥스 포함
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=20)

# Calculation Accuracy

In [20]:
def accuracyCalculation(model, test_loader):
    # 학습을 진행하지 않을 것이므로 torch.no_grad()
    Accuracy = 0
    model = model.eval()
    
    with torch.no_grad():
        for num, data in enumerate(test_loader):
            X_test, Y_test = data
            X_test = X_test.to(device)
            Y_test = Y_test.to(device)

            prediction = model(X_test)
            correct_prediction = torch.argmax(prediction, 1) == Y_test
            accuracy = correct_prediction.float().mean() 
            Accuracy += accuracy / len(test_loader)

    return Accuracy

# Train

In [ ]:
total_batch = len(train_loader)
bestepoch = 0
bestaccuracy = -1
history = {'loss' : [], 'accuracy' : []}

for epoch in range(training_epochs): 
    avg_cost = 0
    model.train()
    
    for num, data in tqdm(enumerate(train_loader)): 
        X, Y = data
        X = X.to(device) 
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch
        
    accuracy = accuracyCalculation(model, test_loader)
    history['loss'].append(avg_cost.cpu().detach().numpy())
    history['accuracy'].append(accuracy.cpu().detach().numpy())
    
    if accuracy > bestaccuracy:
        torch.save(model, "../weight/Gap_fine8.pth")
        bestaccuracy = accuracy
        bestepoch = epoch
    
    scheduler.step(accuracy) 
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))
    print(f"now Accuracy: {accuracy}")
    print(f"Best Epoch: {bestepoch}, Best Accuracy: {bestaccuracy}")
    
print("Learning Finished")

In [ ]:
num_epochs = training_epochs

# plot loss progress
plt.title("Train Loss")
plt.plot(range(1,num_epochs+1),history['loss'],label='train loss')
plt.ylabel("Loss")
plt.xlabel("Training Epochs")
plt.legend()
plt.show()

# plot accuracy progress
plt.title("Train Accuracy")
plt.plot(range(1,num_epochs+1),history['accuracy'],label='accuracy')
plt.ylabel("Accuracy")
plt.xlabel("Training Epochs")
plt.legend()
plt.show()